# Labsheet 2: Multi-tasking with millis()

We will explore the built in Arduino function `millis()` to allow us to imitate a task scheduling system for running blocks of code.  This is useful if we want to do multiple things at once.  For instance, we may wish to update our motor control every 0.5 seconds, whilst updating our line sensor every 0.1 seconds, and to make sure these happen independently.  It would be no good if our code had to stop and wait every 0.5 seconds - that would mean it could never check the sensors every 0.1 seconds.

The `millis()` function reports the time passed in milliseconds since the arduino was powered up.  We can use this to decide to run a block of code once a period of time in milliseconds has elapsed.  If you wanted a smaller time interval, there is also a `micros()` function which operates in the same way we use `millis()`, but reports microseconds since power-up.

For `millis()` to keep track of time, the arduino has some code running in the background by default.  Behind `millis()`, a peripheral device called a **timer** is independently tracking the passage of time, and running a piece of code called an **interrupt service routine (ISR)** which updates the count for `millis()` automatically.  

Using the built in function millis() means that we do not have to get involved with using a **timer/ISR**.  After this worksheet, you may wish to look at the supplementary labsheet on Interrupts & Timers.  The supplementary labsheet isn't necessary to complete the coursework.  If you want to work with microcontrollers in the future it is worthwhile investigating.  




# Exercise 1: Using Arduino _millis()_ to Multi-task

When you have written code before, you may have used the delay() function or delayMicroseconds() function to get your program to stop and wait.  For instance, you may have read a sensor (or blink an LED) every 10 seconds with code like the following:

```C++
// Remember: loop is called again and again.
void loop() {
  
    int sensor_value = analogRead( A0 );    // Do a sensor read, analog pin 0

    delay( 10000 );                    // loop() is stuck waiting here for 10seconds.
    
}
```

The problem with the above code is that when _delay( 10000 )_ is called, your program cannot do anything else - it will sit waiting for 10000 milliseconds.  Generally, this means your robot would not be able to respond to any new information.  This is referred to as **blocking** code.  

Robots typically have to do many things at once, so it is no good if our program becomes blocked.  Instead, we want to write **non-blocking** code - code that can run when it is necessary, and otherwise let the microcontroller CPU do other tasks.  Blocking and non-blocking refer to the fact that code is **sequential**, the program runs (generally speaking) from top to bottom without any real intelligence for itself.

The arduino _millis()_ function provides how many milliseconds have passed since the arduino was turned on (a timestamp).  The Arduino is configured by default to alway count up in milliseconds as a background task.  How this actually works is that the Timer peripheral _timer0_ is configured to keep track of time for you by using an _interrupt service routine (ISR)_.   It is useful to remember that _timer0_ is always counting up for you in the background from the moment the power is switched on.

_millis()_ provides the number of milliseconds elapsed in the format **_unsigned long_** (more info <a href="https://www.arduino.cc/reference/en/language/variables/data-types/unsignedlong/">here</a>).  This means the Arduino can count up to 4,294,967,295 milliseconds.  That is approximately 50 days.  After 50 days, it will reach the maximum value and begin counting from 0 again, which is called an **overflow**.  This means we can only use _millis()_ as a reliable mechanism for 50 days - it is not a perfect solution.  However, your Romi will probably run out of battery power before _millis()_ finished a complete count cycle.  

We can use _millis()_ to keep track of how much time has passed by recording the millisecond count from one time to another - we will refer to these as **timestamps**.  Note that _millis()_ is relative time - how much time has passed, **not the actual time of day**.  We therefore compare two values reported by _millis()_ to work out how much time has passed.  

<img src="https://github.com/paulodowd/EMATM0054_20_21/blob/master/images/timestamps.png?raw=true"/>


By encapsulating a block of code within an *if...* statement, we can ensure it only operates when a certain amount of time has elapsed.  Note that your code is still _sequential_, and we are making a clever use of _selection_.

The following code provides a complete working example:

```C++
unsigned long last_timestamp;   // We will store a timestamp in this.
int sensor_value;             // We will store a sensor reading in this.

// Remember: Setup only runs once when the arduino is powered up.
void setup() {

  pinMode(A0, INPUT );        // Setup up A0 as input to read.

  last_timestamp = millis();    // We set an intial timestamp value.


}


// Remember: loop is called again and again.
void loop() {

    // Get how much time has passed right now.
    unsigned long time_now = millis();     

    // Work out how many milliseconds have gone passed by subtracting
    // our two timestamps.  time_now will always be bigger than the
    // time_of_read (except when millis() overflows after 50 days).
    unsigned long elapsed_time = time_now - last_timestamp;



    // See if 10000 milliseconds have elapsed
    // If not, this block is skipped.
    if( elapsed_time > 10000 ) {

        // Since 10000ms elapsed, we overwrite our last_timestamp with 
        // the new current time so that another 10000ms is needed to pass.
        // !! NOT RESETING THE TIME STAMP IS AN EXTREMELY COMMON BUG !!
        last_timestamp = millis();

        // Do our sensor read.
        sensor_value = analogRead( A0 );

    }

    // Code outside the above if{} will run on every loop!
    // Therefore code here is no longer stopped waiting for a delay()

}

```


### Tasks
- Extend the example code to include multiple *if...* blocks  to flash the three LED's on the Romi independently at different time intervals.  
    - Change the pinMode() call to set the three LED pins to outputs.
    - Change the analogRead() call to a digitalWrite() for your LED pins.
    - **Hint:** You can find the code to setup and flash an LED (a digital pin) within **Lab Sheet 1: Meet the Romi**. 
    - **Hint:** you will need extra _unsigned long_ timestamps variables declared globally to keep track activaty different _if..._ statement blocks.  Give them meaningful names (e.g. unsigned long red_led_ts;).  It is common to use global variables with microcontrollers, and when you compile your code you should be able to see exactly how much system memory you code will use.
    - **Hint:**  You can find which pins the 3 LED's are on using the table <a href="https://www.pololu.com/docs/0J69/3.8">here</a>.  For the pin number to use in your code, look in the column labelled 'Arduino Pin Names'.  
    - **Note:** You may notice that the LED labelled 13 is **on** when the LED's labelled TX & RX are **off**, and vice-versa.  This is because the electronics are wired in opposite ways for these LED's.  
    - **Note:** If you use the **Serial()** functions , this will operate the TX and RX LED's, making them blink out of your control.  It is common for many devices to share pins on a microcontroller.
    
    

# Exercise 2: Using the Buzzer & millis() or micros()

In this exercise we'll configure the Romi to activate the audio buzzer.  In labsheet 1 we used _analogWrite()_ to activate the buzzer.  _analogWrite()_ creates a voltage level by switching a digital pin on and off very fast - called pulse width modulation (PWM).  With analogWrite(), we effectively control the voltage level, but not the frequency (frequency is roughly how fast we are turning it on and off).  Therefore, your buzzer could only beep with one tone, but you were controlling the volume (the power).  

The frequency at which the buzzer is turned on-and-off will determine the pitch of sound you hear.  We can write our own PWM functionality by using _digitalWrite()_ to switch between on and off (1/0), and we can use _millis()_ or _micros()_  to control the frequency.  

**Note that** it is very easy to create tones that the human ear cannot detect, or that we will hear as strange noises.  Whilst working on this labsheet, cats, dogs, bats and other animals may decide they do not like you.

The following code provides a skeleton template for you to generate sounds of different pitch:


```C++
// Setting a #define is like a find-and-replace
#define BUZZER_PIN ????

// Global variables.


// Remember, setup runs once when the romi is powered up
void setup() {
  

  //Start a serial connection
  Serial.begin(9600);
  // Wait for stable connection, report reset.
  delay(1000);
  Serial.println("***RESET***");


  // Set pin 6 (buzzer) to output.
  pinMode( BUZZER_PIN, OUTPUT);

  // Initialise your global variables.

}

// Remember, loop repeats again and again.
void loop() {


  // Implement a millis() or micros() task block
  // to toggle the state of the buzzer.

}
```


- <a href="https://www.pololu.com/docs/0J69/3.8">Find out which pin the piezo buzzer is attached to</a>, add this to the above code.

- Refering to the previous exercise, implement a _millis()_ task block to toggle the state of your buzzer.  Avoid using an iterator (like for(), while()) within _loop()_.
- Explore values of buzz_interval to find different sounds.
- Update the code to use micros() rather than millis() to achieve more pitches of sound.
- **Hint**: you'll need to create some global variables to remember the state and timestamp for your buzzer routine between _loop()_ executions.
- To stop your course mates going mad, you may wish to stick a piece of bluetack, plastacine, or tape over the hole of the buzzer on your Romi to make it quieter.  To locate the buzzer on your Romi, locate the part making the annoying noises.


# Exercise 3: 


- Write an _if()_ statement into your loop so that the the value of buzz_interval increments every 1000 milliseconds.  Done correctly, you should hear the beeping going up in pitch every second.
- **Hint:** You can use another timestamp variable and _if()_ block to run every 1000 milliseconds, seperate from your buzzer updating block.  Essentially, running two tasks at two different time intervals.

    
# Exercise 4:

- Use an array to create a list of specific time interval values for your buzzer, selected by you.  Essentially, a list of different buzzing tones.  Update your code to move through and refer to this array.  Move through your array at 1 second intervals.  
- See if you can make your Romi play a tune you recognise.
- **Hint**: Remember to check for indexing beyond the length of your array.
- **Hint**: If you've not used arrays in C before, here is some example syntax:


```C++
// Global scope
unsigned long intervals[6] = { 1000, 500, 100, 600, 200, 500 };
int interval_index;

// ... elsewhere in code

  interval_index = 0;   // here, using position 0 in array.
  buzz_interval = intervals[ interval_index ];    // read array position.

// ... code continues.
```

# Exercise 5:

- Take the code you have experimented with in this Labsheet and see if you can encapsulate it within a function.  The aim being, that you can make a call to your function to play a tune.
- See if you can implement your function without the use of iteration (such as a for() or while() loop), so that your _loop()_ execution is not blocked.
- Just for fun, you may want to make this your 'Victory Music', which your Romi will be able to play when you successfully complete the Line Following Challenge.